In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import UpdaterMD
from mindsponge.control import VelocityVerlet, Langevin
from mindsponge.callback import WriteH5MD, RunInfo

[WARNING] ME(311912:139923945248000,MainProcess):2023-03-25-16:44:24.582.318 [mindspore/run_check/_check_version.py:75] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")

## 使用力场模板

MindSPONGE中支持yaml格式的力场参数信息配置，其中大体分为`模板`和`参数`两个大类。`模板`中主要存储系统中的原子名称、原子类型、原子质量、原子序数、成键关系等基本信息，也是一般都不需要去改动的信息。`参数`中顾名思义就是存储一些力场参数，比如键长相互作用$E_b=k_b(l-l_0)^2$中的$(k_b,l_0)$就是一组力场参数。不同的原子组合形成的键、角度、二面角还有其他的一些远程相互作用等，都会对应于不同的参数，在不同的力场中千差万别。

比如这是一个三点水的模板配置信息：

```yaml
template:
  WAT:
    atom_name: [O, H1, H2]
    atom_type: [OW, HW, HW]
    atom_mass: [16.00, 1.008, 1.008]
    atomic_number: [8, 1, 1]
    bond:
    - [0, 1]
    - [0, 2]
    head_atom: null
    tail_atom: null
```

以及一个三点水的力场参数信息：

```yaml
template:
  base: water.tip3p.yaml
parameters:
  bond_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [bond_length, force_constant]
    parameters:
      OW-HW: [0.09572, 502416]
  angle_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [bond_angle, force_constant]
    parameters:
      HW-OW-HW: [1.824218, 628.02]
  coulomb_energy:
    length_unit: nm
    energy_unit: kj/mol
  vdw_energy:
    length_unit: nm
    energy_unit: kj/mol
    parameter_names:
      atoms: [sigma, epsilon]
    parameters:
      OW: [0.315061, 0.636386]
      HW: [0.0, 0.0]
```

用户可以根据不同的力场自定义一个这样的yaml文件，然后就可以在MindSPONGE平台上非常方便的进行分子动力学模拟。

In [3]:
system = Molecule(template='water.tip3p.yaml')

In [4]:
potential = ForceField(system, parameters='TIP3P')

In [5]:
opt = UpdaterMD(system,
                integrator=VelocityVerlet(system),
                thermostat=Langevin(system, 300),
                time_step=1e-3)

In [6]:
md = Sponge(system, potential, opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [7]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b02.h5md', save_freq=10, write_velocity=True, write_force=True)

In [8]:
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-25 16:44:36
[MindSPONGE] Step: 0, E_pot: 9.203461, E_kin: 0.0, E_tot: 9.203461, Temperature: 0.0
[MindSPONGE] Step: 10, E_pot: 1.1315365, E_kin: 5.2428923, E_tot: 6.3744287, Temperature: 210.19167
[MindSPONGE] Step: 20, E_pot: 3.115917, E_kin: 5.5845838, E_tot: 8.7005005, Temperature: 223.89035
[MindSPONGE] Step: 30, E_pot: 5.0809994, E_kin: 4.040018, E_tot: 9.121017, Temperature: 161.9675
[MindSPONGE] Step: 40, E_pot: 0.2771599, E_kin: 7.0890293, E_tot: 7.366189, Temperature: 284.20474
[MindSPONGE] Step: 50, E_pot: 2.061215, E_kin: 5.700026, E_tot: 7.761241, Temperature: 228.51852
[MindSPONGE] Step: 60, E_pot: 2.2719703, E_kin: 3.269556, E_tot: 5.5415263, Temperature: 131.07907
[MindSPONGE] Step: 70, E_pot: 3.2417796, E_kin: 3.2043552, E_tot: 6.4461346, Temperature: 128.46512
[MindSPONGE] Step: 80, E_pot: 1.85303, E_kin: 2.5436788, E_tot: 4.3967085, Temperature: 101.97808
[MindSPONGE] Step: 90, E_pot: 0.35244513, E_kin: 1.7013304, E_tot: 2.05

![](../../docs/tutorial_b02.gif)